In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import torch
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from skimage.transform import resize

In [ ]:
from params import *

## Data

### Load

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')

In [ ]:
df_train.dropna(inplace=True)

In [ ]:
frame_impacts = df_train[["image_name", "extended_impact"]].groupby('image_name').max().reset_index()
frame_impacts = frame_impacts.rename(columns={"extended_impact": "frame_has_impact"})
df_train = df_train.merge(frame_impacts, on="image_name")
# df_train = df_train[df_train['frame_has_impact'] == 1]

###  Aux target

In [ ]:
# Single-label approach
aux_label = np.zeros((len(df_train)))

aux_label[df_train['extended_Helmet'] == 1] = 1
aux_label[df_train['extended_Shoulder'] == 1] = 2
aux_label[df_train['extended_shoulder'] == 1] = 2
aux_label[df_train['extended_Body'] == 1] = 2
aux_label[df_train['extended_Hand'] == 1] = 2
aux_label[df_train['extended_Ground'] == 1] = 3

Counter(aux_label)

In [ ]:
df_train['aux_target'] = list(aux_label)

In [ ]:
sns.countplot(df_train['impactType'])
plt.yscale('log')
plt.show()

### Folds

In [ ]:
folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")

## Dataset

In [ ]:
from data.dataset import NFLDatasetCls
from data.transforms import get_transfos_cls

In [ ]:
dataset = NFLDatasetCls(
    df_train.copy(),
    transforms=get_transfos_cls(visualize=True, train=True),
    root=CROP_PATH,
    target_name='extended_impact'
)

In [ ]:
i = np.random.choice(len(dataset))

for i in np.random.choice(len(dataset), 100):
    image, y, y_aux = dataset[i]
    
    if y:
        plt.figure(figsize=(4, 4))
        plt.imshow(image.numpy().transpose(1, 2, 0).copy())
        plt.title(f'Sample #{i} - y={y} - y_aux={y_aux}')
        plt.axis(False)
        plt.show()

## Model

In [ ]:
from model_zoo.models_cls_3d import get_model_cls_3d

In [ ]:
# model = get_model_cls('efficientnet-b0', num_classes=1, num_classes_aux=5)
model = get_model_cls_3d('resnet18', num_classes=1, num_classes_aux=0)

In [ ]:
x = torch.randn(1, 3, 9, 64, 64)

In [ ]:
y = model(x)

In [ ]:
y[0].size()

In [ ]:
dataset = NFLDatasetCls(
    df_train.copy(),
    transforms=get_transfos_cls(visualize=False, train=True),
    root=CROP_PATH,
)

In [ ]:
image, y, y_aux = dataset[0]

In [ ]:
pred = model(
    image.unsqueeze(0),
)

## Training

In [ ]:
from training.main_cls import k_fold_cls

In [ ]:
from utils.logger import prepare_log_folder, save_config, create_logger

In [ ]:
BATCH_SIZES = {
    "resnet18": 256,  # retrain ?
    "resnet34": 256,  # retrain ?
    "resnext50_32x4d": 128,  # 256 ?
    "efficientnet-b1": 128,
    "efficientnet-b2": 128,
    "efficientnet-b3": 128,
    "efficientnet-b4": 64,
}

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    target_name = "extended_impact"
    seed = 42
    verbose = 1
    img_path = CROP_PATH
    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True

    # k-fold
    k = 5
    random_state = 0
    selected_folds = [0] #, 1, 2, 3, 4]

    # Model
    name = "efficientnet-b3"  #"efficientnet-b3", "resnet18", "resnet34"
    num_classes = 1
    
    aux_mode = "softmax"  #sigmoid
    
#     num_classes_aux = 4 if aux_mode == "softmax" else 3
    num_classes_aux = 0

    # Training       
    batch_size = BATCH_SIZES[name]
    samples_per_player = int(5 / 64 * batch_size)
    samples_per_player = 0
    optimizer = "Adam"
    
    acc_steps = 1
    epochs = 20 if samples_per_player else 4
    swa_first_epoch = 15

    lr = 1e-3  # 5e-4 / 1e-3
    warmup_prop = 0.05
    val_bs = batch_size * 2
    
    first_epoch_eval = 0


In [ ]:
DEBUG = False
log_folder = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH_CLS)
    print(f'Logging results to {log_folder}')
    config_df = save_config(Config, log_folder + 'config.json')
    create_logger(directory=log_folder, name="logs.txt")

pred_oof = k_fold_cls(
    Config,
    df_train,
    log_folder=log_folder
)